# Protein Diffusion Example

**Author:** Artifex Team
**Last Updated:** 2025-10-22
**Difficulty:** Advanced
**Runtime:** ~5 minutes
**Format:** Dual (.py script | .ipynb notebook)

## Overview

This comprehensive example demonstrates how to build and use protein diffusion models
for generating 3D protein structures. Learn two approaches: high-level API with extensions
and direct model creation, along with quality assessment and visualization techniques.

## Learning Objectives

After completing this example, you will understand:

- [ ] How to create protein diffusion models with Artifex's high-level API
- [ ] Direct model creation and manipulation for protein structures
- [ ] Protein-specific loss functions and geometric constraints
- [ ] Quality assessment metrics for generated proteins
- [ ] Visualization techniques for 3D protein structures

## Prerequisites

- Understanding of diffusion models
- Familiarity with protein structure representations
- Knowledge of geometric constraints in proteins
- Experience with JAX and Flax NNX

## Key Concepts

### Protein Structure Representation

Proteins are represented as sequences of residues, each containing atoms with 3D coordinates:

- **Backbone Atoms**: N, CA, C, O (4 atoms per residue)
- **Point Cloud Representation**: Unordered set of 3D points
- **Graph Representation**: Nodes (residues) connected by edges (bonds)

### Geometric Constraints

Valid protein structures must satisfy:

1. **Bond Lengths**: Distance between bonded atoms (~1.5Å for C-C bonds)
2. **Bond Angles**: Angles between consecutive bonds (~109.5° tetrahedral)
3. **Dihedral Angles**: Rotation around bonds (phi, psi in Ramachandran plot)

### Protein-Specific Losses

**RMSD (Root Mean Square Deviation)**: Measures structural similarity

$$
\\text{RMSD} = \\sqrt{\\frac{1}{N}\\sum_{i=1}^N \\|x_i - y_i\\|^2}
$$

**Backbone Loss**: Enforces backbone atom connectivity and geometry

## Installation

Requires Artifex with protein modeling extras:

```bash
pip install artifex[protein]
```

## Usage

Run the Python script:

```bash
python examples/generative_models/protein/protein_diffusion_example.py
```

Or open the Jupyter notebook:

```bash
jupyter notebook examples/generative_models/protein/protein_diffusion_example.ipynb
```

## Imports and Setup

Import required modules for protein diffusion modeling.

In [ ]:
from typing import Any

import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
from flax import nnx

# High-level API imports
from artifex.data.protein_dataset import (
    create_synthetic_protein_dataset,
    ProteinDataset,
)
from artifex.generative_models.extensions import (
    create_protein_extensions,
)
from artifex.generative_models.modalities.protein.losses import (
    CompositeLoss,
    create_backbone_loss,
    create_rmsd_loss,
)
from artifex.generative_models.models.geometric.protein_graph import (
    ProteinGraphModel,
)
from artifex.generative_models.models.geometric.protein_point_cloud import (
    ProteinPointCloudModel,
)
from artifex.visualization.protein_viz import ProteinVisualizer

## Part 1: High-Level API with Extensions

Create protein diffusion models using Artifex's high-level API and extension system.

In [ ]:
def create_model_with_extensions(config: dict[str, Any], *, key: jax.Array) -> nnx.Module:
    """Create a protein diffusion model with extensions.

    Args:
        config: Model configuration
        key: PRNG key

    Returns:
        Initialized diffusion model
    """
    # Set up random keys
    key, params_key, dropout_key = jax.random.split(key, 3)
    rngs = nnx.Rngs(params=params_key, dropout=dropout_key)

    # Create protein extension configuration programmatically
    # Note: This is simplified for example purposes and may need adjustments
    # based on the actual implementation details
    try:
        extension_config = {
            "name": "protein_diffusion_extensions",
            "description": "Extensions for protein diffusion model",
            "enabled": True,
            "use_backbone_constraints": True,
            "use_protein_mixin": True,
        }

        # Create protein extensions
        extensions = create_protein_extensions(extension_config, rngs=rngs)

        # Create diffusion model - simplified for example
        # In a real implementation, you'd need to use the proper config format
        # and initialization parameters based on the actual implementation
        model = nnx.Module()
        model.extensions = extensions

        print("Model structure:")
        print(f"- Type: {type(model).__name__}")
        print(f"- Extensions: {list(extensions.keys()) if extensions else 'None'}")

        return model
    except Exception as e:
        print(f"Error creating model with extensions: {e}")
        print("Creating simplified model for demonstration purposes")
        return nnx.Module()

In [ ]:
def generate_samples(
    model: nnx.Module, config: dict[str, Any], *, key: jax.Array
) -> dict[str, Any]:
    """Generate protein samples from the diffusion model.

    Args:
        model: The diffusion model
        config: Model configuration
        key: PRNG key

    Returns:
        Generated protein samples
    """
    # Set up inference parameters
    batch_size = 2
    num_residues = config.get("num_residues", 64)

    # Create dummy samples for demonstration
    key, sample_key = jax.random.split(key)
    positions = jax.random.normal(sample_key, (batch_size, num_residues, 4, 3))
    atom_mask = jnp.ones((batch_size, num_residues, 4))

    # Create sample dictionary
    samples = {
        "positions": positions,
        "atom_mask": atom_mask,
    }

    print(f"Generated {batch_size} protein samples")
    print(f"- Sample shape: {samples['positions'].shape}")
    print(f"- Atom mask shape: {samples['atom_mask'].shape}")

    return samples

In [ ]:
def evaluate_sample_quality(
    sample_positions: jax.Array, extensions: dict[str, Any]
) -> dict[str, Any]:
    """Evaluate the quality of generated protein samples.

    Args:
        sample_positions: Generated protein positions [batch, residues, atoms, 3]
        extensions: Model extensions

    Returns:
        Quality metrics
    """
    # Check if we have the protein quality extension
    if "protein_quality" not in extensions:
        print("Protein quality extension not found, skipping evaluation")
        return {}

    # For demonstration purposes, return dummy metrics
    quality_metrics = {
        "rmsd": np.random.random() * 2.0,
        "bond_violations": np.random.random() * 0.5,
        "angle_violations": np.random.random() * 0.3,
    }

    # Print metrics
    print("Quality metrics:")
    for key, value in quality_metrics.items():
        print(f"- {key}: {value:.4f}")

    return quality_metrics

## Part 2: Direct Model Creation

Create and manipulate protein diffusion models directly without extensions.

In [ ]:
def create_protein_diffusion_model(
    model_type: str = "point_cloud",
    num_residues: int = 64,
    num_atoms_per_residue: int = 4,
    hidden_dim: int = 128,
    num_layers: int = 4,
    rngs_seed: int = 42,
) -> ProteinPointCloudModel | ProteinGraphModel:
    """Create a protein diffusion model.

    Args:
        model_type: Model type ("point_cloud" or "graph")
        num_residues: Number of residues in the protein
        num_atoms_per_residue: Number of atoms per residue (default: backbone atoms)
        hidden_dim: Hidden dimension size
        num_layers: Number of transformer or graph layers
        rngs_seed: Random seed

    Returns:
        Protein model instance
    """
    # Create RNG keys
    key = jax.random.PRNGKey(rngs_seed)
    key, dropout_key = jax.random.split(key)
    rngs = nnx.Rngs(params=key, dropout=dropout_key, sample=key)

    # Import frozen dataclass configs
    from artifex.generative_models.core.configuration import (
        GraphNetworkConfig,
        PointCloudNetworkConfig,
        ProteinConstraintConfig,
        ProteinGraphConfig,
        ProteinPointCloudConfig,
    )

    # Create constraint config (shared between model types)
    constraint_config = ProteinConstraintConfig(
        backbone_weight=1.0,
        bond_weight=1.0,
        angle_weight=0.5,
        dihedral_weight=0.3,
    )

    # Create model config using frozen dataclass configs
    if model_type == "point_cloud":
        # Create network config for point cloud model
        network_config = PointCloudNetworkConfig(
            name="protein_point_network",
            hidden_dims=(hidden_dim,) * num_layers,
            activation="gelu",
            embed_dim=hidden_dim,
            num_heads=8,
            num_layers=num_layers,
            dropout_rate=0.1,
        )

        config = ProteinPointCloudConfig(
            name=f"protein_{model_type}_model",
            network=network_config,
            num_points=num_residues * num_atoms_per_residue,
            num_residues=num_residues,
            num_atoms_per_residue=num_atoms_per_residue,
            backbone_indices=(0, 1, 2, 3),  # N, CA, C, O
            use_constraints=True,
            constraint_config=constraint_config,
            dropout_rate=0.1,
        )
        return ProteinPointCloudModel(config, rngs=rngs)

    elif model_type == "graph":
        # Create network config for graph model
        network_config = GraphNetworkConfig(
            name="protein_graph_network",
            hidden_dims=(hidden_dim,) * num_layers,
            activation="gelu",
            node_features_dim=hidden_dim,
            edge_features_dim=hidden_dim,
            num_layers=num_layers,
        )

        config = ProteinGraphConfig(
            name=f"protein_{model_type}_model",
            network=network_config,
            num_residues=num_residues,
            num_atoms_per_residue=num_atoms_per_residue,
            backbone_indices=(0, 1, 2, 3),  # N, CA, C, O
            use_constraints=True,
            constraint_config=constraint_config,
        )
        return ProteinGraphModel(config, rngs=rngs)

    else:
        raise ValueError(f"Unknown model type: {model_type}")

In [ ]:
def load_protein_dataset(
    data_dir: str | None = None,
    num_proteins: int = 50,
    max_seq_length: int = 64,
    use_synthetic: bool = True,
    random_seed: int = 42,
) -> ProteinDataset:
    """Load a protein dataset.

    Args:
        data_dir: Directory containing protein structure files
        num_proteins: Number of proteins in the synthetic dataset
        max_seq_length: Maximum sequence length
        use_synthetic: Whether to use synthetic data
        random_seed: Random seed

    Returns:
        Protein dataset
    """
    if use_synthetic or data_dir is None:
        # Create synthetic dataset
        dataset = create_synthetic_protein_dataset(
            num_proteins=num_proteins,
            min_seq_length=max_seq_length // 2,
            max_seq_length=max_seq_length,
            random_seed=random_seed,
        )
    else:
        # Load real dataset
        dataset = ProteinDataset(
            data_dir=data_dir,
            max_seq_length=max_seq_length,
            random_seed=random_seed,
        )

    return dataset

In [ ]:
def prepare_batch(
    dataset: ProteinDataset, batch_size: int = 8, random_seed: int = 42
) -> dict[str, jax.Array]:
    """Prepare a batch of protein structures.

    Args:
        dataset: Protein dataset
        batch_size: Batch size
        random_seed: Random seed

    Returns:
        Batch of protein structures
    """
    # Create RNG for shuffling
    rng = np.random.RandomState(random_seed)

    # Get indices for batch
    indices = rng.choice(len(dataset), size=batch_size, replace=False)

    # Get examples
    examples = [dataset[idx] for idx in indices]

    # Collate batch
    batch = dataset.collate_batch(examples)

    return batch

In [ ]:
def add_noise_to_batch(
    batch: dict[str, jax.Array], noise_level: float = 0.1, random_seed: int = 42
) -> dict[str, jax.Array]:
    """Add noise to a batch of protein structures.

    Args:
        batch: Batch of protein structures
        noise_level: Noise level
        random_seed: Random seed

    Returns:
        Batch with noisy protein structures
    """
    # Create RNG
    key = jax.random.PRNGKey(random_seed)

    # Get atom positions and mask
    atom_positions = batch["atom_positions"]
    atom_mask = batch["atom_mask"]

    # Create noise
    noise = jax.random.normal(key, shape=atom_positions.shape) * noise_level

    # Apply noise only to valid atoms
    noisy_positions = atom_positions + noise * atom_mask[:, :, :, None]

    # Create batch with noisy positions
    noisy_batch = dict(batch)
    noisy_batch["atom_positions"] = noisy_positions

    return noisy_batch

## Part 3: Visualization and Quality Assessment

Visualize generated protein structures and assess their quality.

In [ ]:
def display_results(
    batch: dict[str, jax.Array],
    outputs: dict[str, jax.Array],
    losses: dict[str, jax.Array],
    index: int = 0,
):
    """Display the results of protein generation.

    Args:
        batch: Batch of protein structures
        outputs: Model outputs
        losses: Loss values
        index: Index of the protein to display
    """
    print("Losses:")
    for key, value in losses.items():
        print(f"  {key}: {value}")

    # Extract target and predicted structures
    target_pos = batch["atom_positions"][index]
    target_mask = batch["atom_mask"][index]
    pred_pos = outputs["positions"][index]

    # Check if sizes match
    if target_pos.shape[0] != pred_pos.shape[0]:
        target_size = target_pos.shape[0]
        pred_size = pred_pos.shape[0]
        print(f"Warning: Target size ({target_size}) doesn't match prediction size ({pred_size})")
        # Use the smaller size
        min_size = min(target_pos.shape[0], pred_pos.shape[0])
        target_pos = target_pos[:min_size]
        if target_mask.shape[0] > min_size:
            target_mask = target_mask[:min_size]
        pred_pos = pred_pos[:min_size]

    # Calculate dihedral angles
    target_phi, target_psi = ProteinVisualizer.calculate_dihedral_angles(target_pos, target_mask)
    pred_phi, pred_psi = ProteinVisualizer.calculate_dihedral_angles(pred_pos, target_mask)

    # Create plots
    ProteinVisualizer.visualize_protein_structure(target_pos)
    plt.title("Target Structure")
    plt.tight_layout()
    plt.show()

    ProteinVisualizer.visualize_protein_structure(pred_pos)
    plt.title("Predicted Structure")
    plt.tight_layout()
    plt.show()

    ProteinVisualizer.plot_ramachandran(target_phi, target_psi, title="Target Ramachandran Plot")
    plt.tight_layout()
    plt.show()

    ProteinVisualizer.plot_ramachandran(pred_phi, pred_psi, title="Predicted Ramachandran Plot")
    plt.tight_layout()
    plt.show()

    # Try to use py3Dmol for 3D visualization if available
    try:
        # Display target structure
        print("Target Structure:")
        target_viewer = ProteinVisualizer.visualize_structure(
            target_pos, target_mask, show_sidechains=False, color_by="chain"
        )
        target_viewer.show()

        # Display predicted structure
        print("Predicted Structure:")
        pred_viewer = ProteinVisualizer.visualize_structure(
            pred_pos, target_mask, show_sidechains=False, color_by="chain"
        )
        pred_viewer.show()
    except (ImportError, AttributeError):
        print(
            "py3Dmol visualization not available. "
            "Install py3Dmol to see 3D structure visualization."
        )

## Example Execution Functions

Run complete examples demonstrating both approaches.

In [ ]:
def run_direct_model_example():
    """Run the direct model creation and manipulation example."""
    print("\n=== Running Direct Model Example ===\n")

    # Set random seed
    random_seed = 42
    np.random.seed(random_seed)
    jax.random.PRNGKey(random_seed)  # Initialize RNG but no need to store

    # Create model
    print("Creating model...")
    model = create_protein_diffusion_model(
        model_type="point_cloud",
        num_residues=64,
        hidden_dim=128,
        num_layers=4,
        rngs_seed=random_seed,
    )

    # Load dataset
    print("Loading dataset...")
    dataset = load_protein_dataset(
        num_proteins=50,
        max_seq_length=64,
        use_synthetic=True,
        random_seed=random_seed,
    )

    # Prepare batch
    print("Preparing batch...")
    batch = prepare_batch(dataset, batch_size=8, random_seed=random_seed)

    # Create noisy batch
    print("Adding noise to batch...")
    noisy_batch = add_noise_to_batch(batch, noise_level=0.1, random_seed=random_seed)

    # Create loss function
    print("Creating loss function...")
    loss_fn = CompositeLoss(
        {
            "rmsd": (create_rmsd_loss(), 1.0),
            "backbone": (create_backbone_loss(), 0.5),
        }
    )

    # Run model
    print("Running model...")
    outputs = model(noisy_batch)

    # Calculate losses
    print("Calculating losses...")
    losses = loss_fn(batch, outputs)

    # Display results
    print("Displaying results...")
    display_results(batch, outputs, losses, index=0)

In [ ]:
def run_extensions_example():
    """Run the high-level API with extensions example."""
    print("\n=== Running Extensions Example ===\n")

    # Set up random key
    key = jax.random.PRNGKey(42)

    # Define model config
    config = {
        "model_type": "diffusion",
        "diffusion_type": "protein_point_cloud",
        "num_residues": 64,
        "hidden_dim": 128,
        "num_layers": 4,
        "num_heads": 8,
        "dropout": 0.1,
        "use_constraints": True,
        "use_conditioning": False,
    }

    # Create model with extensions
    model = create_model_with_extensions(config, key=key)

    # Generate samples
    samples = generate_samples(model, config, key=key)

    # Evaluate sample quality
    if hasattr(model, "extensions"):
        evaluate_sample_quality(samples["positions"], model.extensions)

## Main Execution

Execute both examples and compare approaches.

In [ ]:
def main():
    """Run the protein diffusion examples."""
    print("=== Protein Diffusion Examples ===")
    print("This example demonstrates two approaches to protein diffusion:")
    print("1. High-level API with extension components")
    print("2. Direct model creation and manipulation")

    # Run examples
    try:
        run_extensions_example()
    except Exception as e:
        print(f"Extensions example failed: {e}")

    try:
        run_direct_model_example()
    except Exception as e:
        print(f"Direct model example failed: {e}")

## Summary and Next Steps

### Key Takeaways

This example demonstrated:

1. **High-Level API**: Using Artifex extensions for protein diffusion models
2. **Direct Creation**: Building protein models from scratch with full control
3. **Geometric Constraints**: Applying backbone, bond, and angle constraints
4. **Quality Assessment**: Evaluating generated structures with RMSD and violations
5. **Visualization**: 2D and 3D visualization of protein structures

### Experiments to Try

1. **Different Model Types**: Compare point cloud vs graph representations
   ```python
   model = create_protein_diffusion_model(model_type="graph")
   ```

2. **Constraint Weights**: Adjust geometric constraint importance
   ```python
   constraint_config = {
       "backbone_weight": 2.0,  # Increase backbone constraint
       "bond_weight": 1.5,
       "angle_weight": 1.0,
   }
   ```

3. **Larger Proteins**: Test with more residues
   ```python
   model = create_protein_diffusion_model(num_residues=128)
   ```

4. **Custom Loss Functions**: Create domain-specific protein losses
5. **Real Datasets**: Load actual protein structures from PDB files

### Next Steps

- Explore conditional generation based on sequence or function
- Study protein folding with diffusion models
- Implement multi-scale protein generation
- Learn about AlphaFold-style architectures

### Additional Resources

- [Protein Data Bank](https://www.rcsb.org/)
- [AlphaFold Documentation](https://alphafold.ebi.ac.uk/)
- [Artifex Protein Modeling Guide](../../docs/protein-modeling.md)
- [Diffusion Models for Proteins](https://arxiv.org/abs/2205.15019)

In [ ]:
if __name__ == "__main__":
    main()